In [ ]:
#Defina a empresa em uma palavra (O que é mais marcante).

!pip install googletrans==4.0.0-rc1
!pip install deep-translator

from googletrans import Translator
from deep_translator import GoogleTranslator
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixe os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Carregar os dados
file_path = 'C:\\Users\\FabianaKuhlmann\\Downloads\\Defina a empresa em uma palavra (O que é mais marcante).xlsx'  # Caminho correto do arquivo
data = pd.read_excel(file_path, sheet_name='Planilha1')  # Argumento sheet_name separado

# Extraia a coluna de comentários
comments_column = data['Comentário']

# Carregar stopwords em português e adicionar palavras genéricas que deseja ignorar
stop_words = set(stopwords.words('portuguese'))
custom_exclusions = {"empresa", "omie", "gosto", "trabalho", "dia", "sempre"}  # Palavras específicas para excluir
all_exclusions = stop_words.union(custom_exclusions)

# Processar os comentários e coletar palavras
all_words = []
for comment in comments_column:
    if isinstance(comment, str):  # Garantir que o comentário é uma string
        # Tokenizar o texto
        tokens = word_tokenize(comment.lower(), language='portuguese')
        # Remover stopwords e palavras genéricas
        filtered_words = [word for word in tokens if word.isalpha() and word not in all_exclusions]
        all_words.extend(filtered_words)

# Contar as palavras mais frequentes
word_counts = Counter(all_words)

# Mostrar as 5 palavras mais frequentes com seus "pesos"
top_5_words = word_counts.most_common(5)
print("As 5 palavras mais frequentes e seus pesos:")
for word, count in top_5_words:
    print(f"{word}: {count} ocorrências")

# Traduzir palavras para o inglês
try:
    # Usando googletrans como padrão
    translator = Translator()
    translated_words = {word: translator.translate(word, src='pt', dest='en').text for word in all_words}
except Exception as e:
    print(f"googletrans falhou: {e}. Tentando com deep-translator.")
    # Alternativa com deep-translator
    translated_words = {word: GoogleTranslator(source='pt', target='en').translate(word) for word in all_words}

# Substituir palavras por suas traduções
translated_word_text = " ".join([translated_words[word] for word in all_words])

# Criar e configurar a nuvem de palavras traduzida
wordcloud_translated = WordCloud(
    width=800, height=400,
    background_color='white',
    stopwords=None,  # Stopwords já foram tratadas
    colormap='viridis'
).generate(translated_word_text)

# Exibir a nuvem de palavras traduzida
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_translated, interpolation='bilinear')
plt.axis('off')  # Remove os eixos
plt.title("Nuvem de Palavras Traduzida - Comentários")
plt.show()
